In [4]:
import os, sys

import matplotlib.pyplot as plt
import numpy as np
import joblib
import torch
#import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.utils.data
import torchvision.transforms as transforms
from munch import Munch
from PIL import Image
from torch.autograd import Variable
sys.path.append('heatmaps')
import models.inception
import models.resnet

In [8]:
meta_data = joblib.load('/home/gen/ddsm_meta_data.jbl')

In [21]:
for img_name, meta in meta_data['meta'].items():
    for finding in meta:
        if 'ASSESSMENT 5' in finding:
            print img_name
            print finding

cancers/cancer_02-C_0121_1.RIGHT_CC.LJPEG.1.jpg
['ABNORMALITY 1', 'LESION_TYPE MASS SHAPE IRREGULAR MARGINS SPICULATED', 'ASSESSMENT 5', 'SUBTLETY 5', 'PATHOLOGY MALIGNANT', 'TOTAL_OUTLINES 1 ', 'BOUNDARY']
cancers/cancer_02-C_0121_1.RIGHT_MLO.LJPEG.1.jpg
['ABNORMALITY 1', 'LESION_TYPE MASS SHAPE IRREGULAR MARGINS SPICULATED', 'ASSESSMENT 5', 'SUBTLETY 5', 'PATHOLOGY MALIGNANT', 'TOTAL_OUTLINES 1 ', 'BOUNDARY']
cancers/cancer_15-B_3387_1.RIGHT_MLO.LJPEG.1.jpg
['ABNORMALITY 1', 'LESION_TYPE MASS SHAPE IRREGULAR MARGINS MICROLOBULATED-ILL_DEFINED', 'ASSESSMENT 5', 'SUBTLETY 5', 'PATHOLOGY MALIGNANT', 'TOTAL_OUTLINES 1 ', 'BOUNDARY']
cancers/cancer_02-C_0062_1.LEFT_CC.LJPEG.1.jpg
['ABNORMALITY 2', 'LESION_TYPE MASS SHAPE ARCHITECTURAL_DISTORTION MARGINS SPICULATED', 'ASSESSMENT 5', 'SUBTLETY 5', 'PATHOLOGY MALIGNANT', 'TOTAL_OUTLINES 1 ', 'BOUNDARY']
cancers/cancer_09-B_3024_1.LEFT_MLO.LJPEG.1.jpg
['ABNORMALITY 1', 'LESION_TYPE CALCIFICATION TYPE PLEOMORPHIC-FINE_LINEAR_BRANCHING DISTRIBU

In [24]:
unit_labels_per_person = joblib.load('/home/gen/resnet152_3class_layer4_unit_labels.jbl')

In [28]:
unit_labels = {}
for unit, val in unit_labels_per_person['diondra'].items():
    unit_labels[unit] = []
    for key in unit_labels_per_person.keys():
        if unit in unit_labels_per_person[key].keys():
            unit_labels[unit].append(unit_labels_per_person[key][unit])

In [29]:
unit_labels

{u'unit_0003': [u"Some of these look like patterns of microcalcs, but there's other stuff mixed in here too",
  u'Dense tissue - women with denser breasts are more likely to have cancer, but it is not indicative of cancer\r\n\r\nmasses - may be a sign of cancer'],
 u'unit_0031': [u'Normal Breast Tissue - not associated with cancer\r\n\r\nBlood Vessels - not associated with cancer'],
 u'unit_0036': [u'Seems to be capturing the edge texture of a mass.  I think the edges here might be considered "indistinct" in the BI-RADS lexicon but I\'m not 100% sure. ',
  u'Calcifications - (macro and micro: macro are not indicative of cancer while micro can be)'],
 u'unit_0091': [u'Normal Breast Tissue - not associated with cancer\r\n\r\nMasses - can be benign or cancerous, if irregularly-shaped or have blurred edges, can be indicative of cancer'],
 u'unit_0097': [u'Normal Breast Tissue - not associated with cancer'],
 u'unit_0105': [u'N/A'],
 u'unit_0135': [u'Calcifications - (macro and micro: macro

In [1]:
# Cleaned Labels
unit_labels = {u'unit_0003': "Microcalcifications, Dense Tissue",
 u'unit_0031': 'Normal Breast Tissue',
 u'unit_0036': 'Mass with circumscribed margins',
 u'unit_0091': 'edge of a suspicious mass, mass irregularly-shaped',
 u'unit_0097': 'Normal Breast Tissue',
 u'unit_0135': 'Edge of mass',
 u'unit_0175': 'Normal Breast Tissue',
 u'unit_0187': 'Isolated large calcifications',
 u'unit_0175': 'Normal Breast Tissue',
 u'unit_0281': 'arterial calcifications',
 u'unit_0175': 'Normal Breast Tissue',
 u'unit_0344': 'Dense Tissue',
 u'unit_0175': 'Normal Breast Tissue',
 u'unit_0175': 'Normal Breast Tissue',
 u'unit_0175': 'Normal Breast Tissue',
 u'unit_0175': 'Normal Breast Tissue',
 u'unit_0404': 'Normal Breast Tissue',               
 u'unit_0560': 'Mass with calcifications',
 u'unit_0588': 'Masses with spiculations, suspicious for breast cancer',
 u'unit_0594': 'Calcifications and Masses',
 u'unit_0175': 'Normal Breast Tissue',
 u'unit_0175': 'Normal Breast Tissue',
 u'unit_0659': 'Calcifications and Masses',
 u'unit_0717': 'Masses with spiculated edges',
 u'unit_0725': 'suspicious calcifications',
 u'unit_0820': 'Mass',
 u'unit_0842': 'Normal Breast Tissue',
 u'unit_0860': 'Benign vascular calcifications',
 u'unit_0889': 'Mass',
 u'unit_1001': 'Calcified vessels',
 u'unit_1024': 'Normal Breast Tissue',
 u'unit_1065': 'Interface of tissue with air or fat or normal tissue',               
 u'unit_1118': 'Normal Breast Tissue',               
 u'unit_1122': 'Pectoralis or inframammary folds',
 u'unit_1152': 'Malignant pleomorphic calcifications',
 u'unit_1168': 'Normal Breast Tissue',
 u'unit_1299': 'Spiculation',
 u'unit_1311': 'Normal Breast Tissue',
 u'unit_1373': 'Masses with calcifications',
 u'unit_1405': 'Benign calcifications, vascular clarification',
 u'unit_1430': 'Calcifications and arterial calcification',
 u'unit_1460': 'Masses with calcification clusters inside, a combination that is very suspicious.',
 u'unit_1468': 'Masses with smooth edges and Macrocalification',
 u'unit_1549': 'Normal Breast Tissue',
 u'unit_1583': 'Edge of the mass, speculations associated with cancer',
 u'unit_1593': 'Normal Breast Tissue',
 u'unit_1597': 'Calcified vessels',
 u'unit_1680': 'Calcified vessels',
 u'unit_1780': 'Edge of breast or with the chest wall',
 u'unit_1814': 'Edge of a mass with circumscribed margins',
 u'unit_1917': 'Chest Wall',
 u'unit_1961': 'masses and calcifications of various types (macro or micro)',
 u'unit_1962': 'Calcifications',
 u'unit_2038': 'Normal Breast Tissue'}

In [5]:
for key, val in unit_labels.items():
    print '{} & {}\\\\'.format(key.split('_')[0]+' '+key.split('_')[1], val)

unit 2038 & Normal Breast Tissue\\
unit 1468 & Masses with smooth edges and Macrocalification\\
unit 1583 & Edge of the mass, speculations associated with cancer\\
unit 0594 & Calcifications and Masses\\
unit 0820 & Mass\\
unit 0281 & arterial calcifications\\
unit 0725 & suspicious calcifications\\
unit 0404 & Normal Breast Tissue\\
unit 1311 & Normal Breast Tissue\\
unit 1299 & Spiculation\\
unit 0560 & Mass with calcifications\\
unit 1373 & Masses with calcifications\\
unit 1168 & Normal Breast Tissue\\
unit 1001 & Calcified vessels\\
unit 1405 & Benign calcifications, vascular clarification\\
unit 1430 & Calcifications and arterial calcification\\
unit 1961 & masses and calcifications of various types (macro or micro)\\
unit 1962 & Calcifications\\
unit 0003 & Microcalcifications, Dense Tissue\\
unit 1549 & Normal Breast Tissue\\
unit 1780 & Edge of breast or with the chest wall\\
unit 1680 & Calcified vessels\\
unit 1152 & Malignant pleomorphic calcifications\\
unit 1118 & Normal 

In [2]:
len(unit_labels.keys())

45

In [11]:
joblib.dump(unit_labels,'cleaned_resnet152_3class_unit_labels.jbl')

['cleaned_resnet152_3class_unit_labels.jbl']